In [22]:
seldon_deployment = """
apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  name: wines-classifier
spec:
  predictors:
  - graph:
      children: []
      implementation: MLFLOW_SERVER
      modelUri: s3://mlops-model-artifact/1/d41dd65f245345b3b409ad8121735f34/artifacts/model
      name: wines-classifier
    name: model-a
    replicas: 1
    traffic: 100
    componentSpecs:
    - spec:
        # We are setting high failureThreshold as installing conda dependencies
        # can take long time and we want to avoid k8s killing the container prematurely
        containers:
        - name: wines-classifier
          livenessProbe:
            initialDelaySeconds: 60
            failureThreshold: 100
            periodSeconds: 5
            successThreshold: 1
            httpGet:
              path: /health/ping
              port: http
              scheme: HTTP
          readinessProbe:
            initialDelaySeconds: 60
            failureThreshold: 100
            periodSeconds: 5
            successThreshold: 1
            httpGet:
              path: /health/ping
              port: http
              scheme: HTTP

"""

In [20]:
import yaml

In [23]:
dep = yaml.safe_load(seldon_deployment)

In [24]:
dep

{'apiVersion': 'machinelearning.seldon.io/v1alpha2',
 'kind': 'SeldonDeployment',
 'metadata': {'name': 'wines-classifier'},
 'spec': {'predictors': [{'graph': {'children': [],
     'implementation': 'MLFLOW_SERVER',
     'modelUri': 's3://mlops-model-artifact/1/d41dd65f245345b3b409ad8121735f34/artifacts/model',
     'name': 'wines-classifier'},
    'name': 'model-a',
    'replicas': 1,
    'traffic': 100,
    'componentSpecs': [{'spec': {'containers': [{'name': 'wines-classifier',
         'livenessProbe': {'initialDelaySeconds': 60,
          'failureThreshold': 100,
          'periodSeconds': 5,
          'successThreshold': 1,
          'httpGet': {'path': '/health/ping',
           'port': 'http',
           'scheme': 'HTTP'}},
         'readinessProbe': {'initialDelaySeconds': 60,
          'failureThreshold': 100,
          'periodSeconds': 5,
          'successThreshold': 1,
          'httpGet': {'path': '/health/ping',
           'port': 'http',
           'scheme': 'HTTP'}}}]

In [27]:
from kubernetes import config, client

In [28]:
config.load_kube_config()

In [29]:
v1 = client.CustomObjectsApi()

In [ ]:
http://a2a2d5f20a12342368355def4eb18716-1626501488.eu-west-1.elb.amazonaws.com:80/seldon/stage/wines-classifier/api/v1.0/predictions

In [59]:
v1.get_namespaced_custom_object("machinelearning.seldon.io", "v1alpha2", "stage", "seldondeployments", )

ApiException: (401)
Reason: Unauthorized
HTTP response headers: HTTPHeaderDict({'Audit-Id': '2dcf9602-d2a2-412e-98d1-a297a01a01d3', 'Cache-Control': 'no-cache, private', 'Content-Type': 'application/json', 'Date': 'Sun, 07 Feb 2021 05:55:03 GMT', 'Content-Length': '129'})
HTTP response body: {"kind":"Status","apiVersion":"v1","metadata":{},"status":"Failure","message":"Unauthorized","reason":"Unauthorized","code":401}



In [52]:
custom_resources = dict(
    group = "machinelearning.seldon.io",
    version = "v1alpha2",
    plural = "seldondeployments",
    )

In [53]:
v1.create_namespaced_custom_object	(**custom_resources, namespace="stage", body=dep)

ApiException: (401)
Reason: Unauthorized
HTTP response headers: HTTPHeaderDict({'Audit-Id': '72e8b7ca-6491-45af-9317-709bdd742ccf', 'Cache-Control': 'no-cache, private', 'Content-Type': 'application/json', 'Date': 'Sun, 07 Feb 2021 05:25:55 GMT', 'Content-Length': '129'})
HTTP response body: {"kind":"Status","apiVersion":"v1","metadata":{},"status":"Failure","message":"Unauthorized","reason":"Unauthorized","code":401}

